In [6]:
#read data
import os

# 指定/tmp目录
tmp_dir = '/root/autodl-tmp'

# 列出所有文件和子目录
contents = os.listdir(tmp_dir)

print(f"/tmp目录下的内容：")
for item in contents:
    print(item)

/tmp目录下的内容：
.autodl
wget-log
download?fileId=c2487ff51a6230ba2ff55f0b5a8bfe8f.1
.ipynb_checkpoints
0724.hdf5
chb01.zip
chb02.zip
chb03.zip
iTransformer-main
ETTh1.csv
ETTm1.csv
chb01
chb05.zip
chb09.zip
chb10.zip
chb13.zip
chb14.zip
chb18.zip
dataprecess.ipynb
chb19.zip
chb02
chb03
chb09
chb10
chb13
chb14


In [1]:
import os
import zipfile

tmp_dir = '/root/autodl-tmp'
target_dir = '/root/autodl-tmp/epli'

# 确保目标目录存在
os.makedirs(target_dir, exist_ok=True)

# 获取所有zip文件并按数字排序
zip_files = sorted(
    [f for f in os.listdir(tmp_dir) if f.startswith('chb') and f.endswith('.zip')],
    key=lambda x: int(x[3:-4])  # 提取chbXX.zip中的数字部分排序
)

# 从chb02.zip开始解压（跳过chb01.zip）
for zip_file in zip_files:
    subject_id = zip_file[:-4]  # 去掉.zip后缀
    subject_dir = os.path.join(target_dir, subject_id)
    
    # 判断编号是否≥2且目标目录不存在（避免重复解压）
    if int(zip_file[3:-4]) >= 2 and not os.path.exists(subject_dir):
        zip_path = os.path.join(tmp_dir, zip_file)
        print(f"正在解压: {zip_file}...")
        
        try:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                # 解压到目标目录下的subject_id子目录
                zip_ref.extractall(subject_dir)
            print(f"解压完成: {zip_file} -> {subject_dir}")
        except Exception as e:
            print(f"解压失败: {zip_file} ({str(e)})")
    else:
        if int(zip_file[3:-4]) >= 2:
            print(f"跳过已存在的: {zip_file} (目标目录 {subject_dir} 已存在)")
        else:
            print(f"跳过: {zip_file} (编号小于2)")

print("所有符合条件的ZIP文件已处理完毕")

# 打印解压结果
print("\n解压结果:")
extracted_dirs = [d for d in os.listdir(target_dir) if os.path.isdir(os.path.join(target_dir, d))]
for d in sorted(extracted_dirs, key=lambda x: int(x[3:])):
    print(f"- {d}")

跳过: chb01.zip (编号小于2)
跳过已存在的: chb02.zip (目标目录 /root/autodl-tmp/epli/chb02 已存在)
跳过已存在的: chb03.zip (目标目录 /root/autodl-tmp/epli/chb03 已存在)
跳过已存在的: chb05.zip (目标目录 /root/autodl-tmp/epli/chb05 已存在)
跳过已存在的: chb09.zip (目标目录 /root/autodl-tmp/epli/chb09 已存在)
跳过已存在的: chb10.zip (目标目录 /root/autodl-tmp/epli/chb10 已存在)
跳过已存在的: chb13.zip (目标目录 /root/autodl-tmp/epli/chb13 已存在)
跳过已存在的: chb14.zip (目标目录 /root/autodl-tmp/epli/chb14 已存在)
跳过已存在的: chb18.zip (目标目录 /root/autodl-tmp/epli/chb18 已存在)
跳过已存在的: chb19.zip (目标目录 /root/autodl-tmp/epli/chb19 已存在)
跳过已存在的: chb20.zip (目标目录 /root/autodl-tmp/epli/chb20 已存在)
跳过已存在的: chb21.zip (目标目录 /root/autodl-tmp/epli/chb21 已存在)
跳过已存在的: chb23.zip (目标目录 /root/autodl-tmp/epli/chb23 已存在)
所有符合条件的ZIP文件已处理完毕

解压结果:
- chb01
- chb02
- chb03
- chb05
- chb09
- chb10
- chb13
- chb14
- chb18
- chb19
- chb20
- chb21
- chb23


In [6]:
import os
import zipfile

# 1. 设置文件路径
zip_file_path = "/root/autodl-tmp/chb05.zip"  # 替换为你的实际路径
extract_dir = "/root/autodl-tmp/epli"    # 解压目标目录

# 2. 确保目标目录存在
os.makedirs(extract_dir, exist_ok=True)

# 3. 解压ZIP文件
try:
    print(f"正在解压 {zip_file_path} 到 {extract_dir}...")
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print("解压成功！")
    
    # 4. 验证解压结果
    extracted_files = os.listdir(extract_dir)
    print(f"解压出 {len(extracted_files)} 个文件:")
    for f in sorted(extracted_files)[:5]:  # 只显示前5个文件
        print(f"  - {f}")
        
except zipfile.BadZipFile:
    print("错误：ZIP文件已损坏！")
    print("尝试修复命令：")
    print(f"zip -FF {zip_file_path} --out {zip_file_path}.fixed")
    
except Exception as e:
    print(f"解压失败: {str(e)}")

正在解压 /root/autodl-tmp/chb05.zip 到 /root/autodl-tmp/epli...
解压失败: Error -3 while decompressing data: invalid stored block lengths


In [7]:
#检查损坏文件chb055完整性
import os
import mne
from tqdm import tqdm

extract_dir = "/root/autodl-tmp/epli/chb05"

def validate_edf_files(folder_path):
    """增强版EDF文件验证"""
    # 获取所有EDF文件（包含子目录）
    edf_files = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith('.edf'):
                edf_files.append(os.path.join(root, file))
    
    if not edf_files:
        print(f"警告: 目录 {folder_path} 中未找到EDF文件")
        return [], []

    valid_files = []
    corrupted_files = []
    
    for file_path in tqdm(edf_files, desc="验证EDF文件"):
        try:
            # 严格模式验证
            raw = mne.io.read_raw_edf(file_path, preload=False, verbose='error')
            
            # 额外检查基本参数
            if len(raw.ch_names) == 0:
                raise ValueError("无有效通道")
            if raw.info['sfreq'] <= 0:
                raise ValueError("无效采样率")
                
            valid_files.append(os.path.basename(file_path))
        except Exception as e:
            corrupted_files.append((os.path.basename(file_path), str(e)))
    
    print(f"\n验证结果（共{len(edf_files)}个文件）：")
    print(f"✅ 有效文件: {len(valid_files)}个")
    print(f"❌ 损坏文件: {len(corrupted_files)}个")
    
    if corrupted_files:
        print("\n损坏文件详情：")
        for file, error in corrupted_files[:5]:  # 最多显示5个错误
            print(f"  - {file}: {error}")
        if len(corrupted_files) > 5:
            print(f"  ...（省略{len(corrupted_files)-5}个错误）")
    
    return valid_files, corrupted_files

# 执行验证
valid, corrupted = validate_edf_files(extract_dir)

# 保存结果
with open(os.path.join(extract_dir, "validation_report.txt"), 'w') as f:
    f.write("有效文件:\n" + "\n".join(valid) + "\n\n")
    f.write("损坏文件:\n" + "\n".join([f"{f[0]}: {f[1]}" for f in corrupted]))

验证EDF文件: 100%|██████████| 23/23 [00:01<00:00, 19.49it/s]


验证结果（共23个文件）：
✅ 有效文件: 23个
❌ 损坏文件: 0个


In [1]:
import os

# 指定/tmp目录
tmp_dir = '/root/autodl-tmp/epli'

# 列出所有文件和子目录
contents = os.listdir(tmp_dir)

print(f"/tmp目录下的内容：")
for item in contents:
    print(item)

/tmp目录下的内容：
chb01
chb02
chb03
chb09
chb10
chb14
chb05
chb13
chb18
chb19
chb20
chb21
chb23


In [9]:
import os

# 指定/tmp目录
tmp_dir = '/root/autodl-tmp/epli/chb02'

# 列出所有文件和子目录
contents = os.listdir(tmp_dir)

print(f"/tmp目录下的内容：")
for item in contents:
    print(item)

/tmp目录下的内容：
chb02-summary.txt
chb02_01.edf
chb02_02.edf
chb02_03.edf
chb02_04.edf
chb02_05.edf
chb02_06.edf
chb02_07.edf
chb02_08.edf
chb02_09.edf
chb02_10.edf
chb02_11.edf
chb02_12.edf
chb02_13.edf
chb02_14.edf
chb02_15.edf
chb02_16+.edf
chb02_16+.edf.seizures
chb02_16.edf
chb02_16.edf.seizures
chb02_17.edf
chb02_18.edf
chb02_19.edf
chb02_19.edf.seizures
chb02_20.edf
chb02_21.edf
chb02_22.edf
chb02_23.edf
chb02_24.edf
chb02_25.edf
chb02_26.edf
chb02_27.edf
chb02_28.edf
chb02_29.edf
chb02_30.edf
chb02_31.edf
chb02_32.edf
chb02_33.edf
chb02_34.edf
chb02_35.edf
index.html


In [10]:
#load-info
import mne
import numpy as np
import pandas as pd
from glob import glob
import os

# 数据路径设置
data_root = '/root/autodl-tmp/epli'
output_dir = '/root/autodl-tmp/processed'

# 创建输出目录
os.makedirs(output_dir, exist_ok=True)

# 获取所有EDF文件路径
def get_edf_paths(subject):
    subject_dir = os.path.join(data_root, f'chb{subject:02d}')
    return sorted(glob(os.path.join(subject_dir, '*.edf')))

# 示例：加载chb02的第一个EDF文件
raw = mne.io.read_raw_edf(get_edf_paths(2)[0], preload=True)
print(f"原始信息:\n{raw.info}")
print(f"通道名称:\n{raw.ch_names}")

Extracting EDF parameters from /root/autodl-tmp/epli/chb02/chb02_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


/tmp/ipykernel_4677/2196780285.py:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(get_edf_paths(2)[0], preload=True)


原始信息:
<Info | 8 non-empty values
 bads: []
 ch_names: FP1-F7, F7-T7, T7-P7, P7-O1, FP1-F3, F3-C3, C3-P3, P3-O1, ...
 chs: 23 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 128.0 Hz
 meas_date: 2074-07-24 15:29:39 UTC
 nchan: 23
 projs: []
 sfreq: 256.0 Hz
 subject_info: 1 item (dict)
>
通道名称:
['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8-0', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8-1']


In [15]:
#channel  choosing （静态）
#extract the target channel directly
#F3-C3, C3-P3, F4-C4, C4-P4, FZ-CZ, CZ-PZ
from scipy.signal import welch
import numpy as np
import mne

# 文献推荐的6个通道
TARGET_CHANNELS = ['F3-C3', 'C3-P3', 'F4-C4', 'C4-P4', 'FZ-CZ', 'CZ-PZ']

def select_channels(raw):
    """
    基于功率谱密度选择最佳通道（静态选择6个候选通道中的最优者）
    """
    # 1. 检查并选择存在的通道
    available_chs = [ch for ch in TARGET_CHANNELS if ch in raw.ch_names]
    if not available_chs:
        raise ValueError("目标通道不存在！")
    
    # 2. 计算各通道的PSD（使用MNE的compute_psd方法）
    psds = raw.copy().pick(available_chs).compute_psd(
        fmin=1, 
        fmax=50, 
        method='welch',
        n_fft=256,
        n_overlap=128
    )
    
    # 3. 计算平均功率并选择最佳通道
    avg_power = psds.get_data().mean(axis=1)  # 平均各频段的功率
    best_ch_idx = np.argmax(avg_power)
    best_ch = available_chs[best_ch_idx]
    
    print("各通道PSD比较:")
    for ch, power in zip(available_chs, avg_power):
        print(f"  {ch}: {power:.2f}")
    print(f"选择通道: {best_ch} (平均功率: {avg_power[best_ch_idx]:.2f})")
    
    return raw.copy().pick([best_ch])

# 使用示例（假设raw是已加载的Raw对象）
selected_raw = select_channels(raw)
print(f"最终选择的通道: {selected_raw.ch_names}")

Effective window size : 1.000 (s)
各通道PSD比较:
  F3-C3: 0.00
  C3-P3: 0.00
  F4-C4: 0.00
  C4-P4: 0.00
  FZ-CZ: 0.00
  CZ-PZ: 0.00
选择通道: C3-P3 (平均功率: 0.00)
最终选择的通道: ['C3-P3']


In [ ]:
###这一块可以做研究，静态通道选择

In [17]:
#验证（验证思路 所有目标通道加总 所有排除通道加总 比较2者）（运行有误，换了一版）
'''import numpy as np
import mne
from glob import glob
import os

# 1. 设置路径
subject_id = '02'  # 分析chb02
data_dir = '/root/autodl-tmp/epli/chb' + subject_id  # 根据实际路径修改
output_dir = '/root/autodl-tmp/processed'
os.makedirs(output_dir, exist_ok=True)

# 2. 获取该被试所有EDF文件
edf_files = sorted(glob(os.path.join(data_dir, '*.edf')))
print(f"找到{len(edf_files)}个EDF文件:")
for f in edf_files:
    print("  ", os.path.basename(f))

# 3. 定义PSD分析函数
def analyze_psd(raw):
    """计算所有通道的PSD均值和标准差"""
    #psds = raw.compute_psd(method='welch', fmin=1, fmax=50, n_fft=256, n_overlap=128)
    psds = raw.compute_psd(method='welch', fmin=1, fmax=50, n_fft=768, n_overlap=128)
    mean_power = psds.get_data().mean(axis=1)  # 各通道平均PSD
    std_power = psds.get_data().std(axis=1)    # 各通道PSD标准差
    return mean_power, std_power, psds.freqs

# 4. 遍历所有EDF文件并统计
all_psd_means = []
all_psd_stds = []
channel_names = None

for edf_file in edf_files:
    raw = mne.io.read_raw_edf(edf_file, preload=True)
    
    # 首次运行时获取通道名称
    if channel_names is None:
        channel_names = raw.ch_names
        print(f"\n所有可用通道 ({len(channel_names)}个):")
        print(channel_names)
    
    # 计算PSD
    mean_power, std_power, freqs = analyze_psd(raw)
    all_psd_means.append(mean_power)
    all_psd_stds.append(std_power)

# 5. 计算全局统计量（跨所有EDF文件）
global_mean = np.mean(all_psd_means, axis=0)
global_std = np.mean(all_psd_stds, axis=0)

# 6. 打印结果（按PSD从高到低排序）
print("\n各通道PSD全局统计（1-50Hz）:")
results = []
for i, ch in enumerate(channel_names):
    results.append({
        'channel': ch,
        'mean_psd': global_mean[i],
        'std_psd': global_std[i],
        'is_target': ch in ['F3-C3', 'C3-P3', 'F4-C4', 'C4-P4', 'FZ-CZ', 'CZ-PZ']
    })

# 按PSD均值排序输出
results_sorted = sorted(results, key=lambda x: -x['mean_psd'])

print("\n{:<10} {:<15} {:<15} {:<10}".format("Channel", "Mean PSD", "Std PSD", "Is Target"))
print("-" * 50)
for item in results_sorted:
    mark = "✓" if item['is_target'] else ""
    print("{:<10} {:<15.2f} {:<15.2f} {:<10}".format(
        item['channel'], item['mean_psd'], item['std_psd'], mark))

# 7. 验证文献结论
target_mask = [ch in ['F3-C3', 'C3-P3', 'F4-C4', 'C4-P4', 'FZ-CZ', 'CZ-PZ'] for ch in channel_names]
print("\n文献目标通道的PSD表现:")
print(f"  - 平均PSD: {np.mean(global_mean[target_mask]):.2f} (非目标通道: {np.mean(global_mean[~np.array(target_mask)]):.2f})")
print(f"  - 稳定性(std): {np.mean(global_std[target_mask]):.2f} (非目标通道: {np.mean(global_std[~np.array(target_mask)]):.2f})")'''

找到36个EDF文件:
   chb02_01.edf
   chb02_02.edf
   chb02_03.edf
   chb02_04.edf
   chb02_05.edf
   chb02_06.edf
   chb02_07.edf
   chb02_08.edf
   chb02_09.edf
   chb02_10.edf
   chb02_11.edf
   chb02_12.edf
   chb02_13.edf
   chb02_14.edf
   chb02_15.edf
   chb02_16+.edf
   chb02_16.edf
   chb02_17.edf
   chb02_18.edf
   chb02_19.edf
   chb02_20.edf
   chb02_21.edf
   chb02_22.edf
   chb02_23.edf
   chb02_24.edf
   chb02_25.edf
   chb02_26.edf
   chb02_27.edf
   chb02_28.edf
   chb02_29.edf
   chb02_30.edf
   chb02_31.edf
   chb02_32.edf
   chb02_33.edf
   chb02_34.edf
   chb02_35.edf
Extracting EDF parameters from /root/autodl-tmp/epli/chb02/chb02_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


/tmp/ipykernel_4677/1071477427.py:34: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=True)



所有可用通道 (23个):
['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8-0', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8-1']
Effective window size : 3.000 (s)
Extracting EDF parameters from /root/autodl-tmp/epli/chb02/chb02_02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


/tmp/ipykernel_4677/1071477427.py:34: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=True)


Effective window size : 3.000 (s)
Extracting EDF parameters from /root/autodl-tmp/epli/chb02/chb02_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


...

Effective window size : 3.000 (s)
Extracting EDF parameters from /root/autodl-tmp/epli/chb02/chb02_34.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


/tmp/ipykernel_4677/1071477427.py:34: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=True)


Effective window size : 3.000 (s)
Extracting EDF parameters from /root/autodl-tmp/epli/chb02/chb02_35.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


/tmp/ipykernel_4677/1071477427.py:34: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=True)


Effective window size : 3.000 (s)

各通道PSD全局统计（1-50Hz）:

Channel    Mean PSD        Std PSD         Is Target 
--------------------------------------------------
CZ-PZ      0.00            0.00            ✓         
FP2-F8     0.00            0.00                      
C4-P4      0.00            0.00            ✓         
P4-O2      0.00            0.00                      
FP2-F4     0.00            0.00                      
FP1-F7     0.00            0.00                      
T7-P7      0.00            0.00                      
P7-T7      0.00            0.00                      
T8-P8-0    0.00            0.00                      
T8-P8-1    0.00            0.00                      
FP1-F3     0.00            0.00                      
C3-P3      0.00            0.00            ✓         
F7-T7      0.00            0.00                      
F8-T8      0.00            0.00                      
P3-O1      0.00            0.00                      
FT10-T8    0.00            0.

In [4]:
#验证方式2  单独检验
import numpy as np
import mne
import pandas as pd
from IPython.display import display

def debug_single_file(edf_path):
    """调试单个EDF文件的处理流程（兼容新版MNE）"""
    print(f"\n{'='*80}\n开始调试文件: {os.path.basename(edf_path)}\n{'='*80}")
    
    try:
        # 1. 加载文件
        raw = mne.io.read_raw_edf(edf_path, preload=True)
        print("文件加载成功！")
        print(f"采样率: {raw.info['sfreq']} Hz")
        print(f"通道数: {len(raw.ch_names)}")
        print(f"原始通道: {raw.ch_names}")

        # 2. 检查重复通道名
        dup_chs = set([ch for ch in raw.ch_names if raw.ch_names.count(ch) > 1])
        if dup_chs:
            print(f"发现重复通道: {dup_chs} (MNE会自动添加后缀)")
        
        # 3. 计算PSD（兼容不同MNE版本）
        try:
            psds = raw.compute_psd(
                method='welch',
                fmin=1,
                fmax=50,
                n_fft=int(3 * raw.info['sfreq']),  # 3秒窗长
                n_overlap=int(1.5 * raw.info['sfreq'])  # 50%重叠
            )
            
            # 获取实际使用的n_fft（新版MNE兼容方案）
            if hasattr(psds, '_get_nfft'):  # 旧版MNE
                n_fft = psds._get_nfft()
            else:  # 新版MNE
                n_fft = psds.get_data().shape[1] * 2 - 1  # 估算n_fft
            
            print(f"PSD计算完成，实际窗长: {n_fft/raw.info['sfreq']:.2f}秒")
        except Exception as e:
            print(f"PSD计算失败: {str(e)}")
            return None

        # 4. 提取特征
        mean_power = psds.get_data().mean(axis=1)
        std_power = psds.get_data().std(axis=1)
        
        # 5. 构建结果表格
        TARGET_CHANNELS = ['F3-C3', 'C3-P3', 'F4-C4', 'C4-P4', 'FZ-CZ', 'CZ-PZ']
        df = pd.DataFrame({
            'Channel': raw.ch_names,
            'Mean_PSD': mean_power,
            'Std_PSD': std_power,
            'Is_Target': ['✓' if ch in TARGET_CHANNELS else '' for ch in raw.ch_names]
        })
        
        # 6. 双版本排序
        df_mean = df.sort_values('Mean_PSD', ascending=False)
        df_std = df.sort_values('Std_PSD', ascending=True)
        
        print("\n按PSD均值降序:")
        display(df_mean.head(10))
        
        print("\n按PSD标准差升序:")
        display(df_std.head(10))
        
        return df
    
    except Exception as e:
        print(f"调试失败: {str(e)}")
        return None

# 使用示例
test_file = '/root/autodl-tmp/epli/chb02/chb02_01.edf'
single_result = debug_single_file(test_file)


开始调试文件: chb02_01.edf
Extracting EDF parameters from /root/autodl-tmp/epli/chb02/chb02_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


/tmp/ipykernel_5461/2209629689.py:12: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_path, preload=True)


文件加载成功！
采样率: 256.0 Hz
通道数: 23
原始通道: ['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8-0', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8-1']
Effective window size : 3.000 (s)
PSD计算完成，实际窗长: 1.15秒

按PSD均值降序:


,Channel,Mean_PSD,Std_PSD,Is_Target
22,T8-P8-1,1.163228e-10,2.723099e-10,
14,T8-P8-0,1.163228e-10,2.723099e-10,
2,T7-P7,9.090949e-11,1.646314e-10,
18,P7-T7,9.090949e-11,1.646314e-10,
19,T7-FT9,8.131234e-11,2.437733e-10,
13,F8-T8,6.895597e-11,1.816730e-10,
0,FP1-F7,6.433853e-11,1.408670e-10,
21,FT10-T8,6.319800e-11,1.538113e-10,
1,F7-T7,5.611117e-11,6.542817e-11,
12,FP2-F8,5.252582e-11,1.327242e-10,



按PSD标准差升序:


,Channel,Mean_PSD,Std_PSD,Is_Target
9,F4-C4,1.212736e-11,2.105498e-11,✓
16,FZ-CZ,8.815153e-12,2.188934e-11,✓
5,F3-C3,1.374809e-11,2.966157e-11,✓
11,P4-O2,2.109663e-11,3.411409e-11,
15,P8-O2,2.425204e-11,3.964897e-11,
17,CZ-PZ,1.664435e-11,5.369907e-11,✓
10,C4-P4,2.356375e-11,5.921558e-11,✓
6,C3-P3,2.419873e-11,6.068463e-11,✓
7,P3-O1,2.197834e-11,6.215666e-11,
4,FP1-F3,3.024333e-11,6.260682e-11,


In [5]:
#批量检验
import numpy as np
import mne
import pandas as pd
import os
from glob import glob
from IPython.display import display

def batch_analyze_channels(subject_id='02'):
    """
    批量处理一个被试的所有EDF文件
    返回所有通道的全局PSD均值和标准差排序结果
    """
    # 1. 设置路径
    data_dir = f'/root/autodl-tmp/epli/chb{subject_id}'
    output_dir = '/root/autodl-tmp/processed'
    os.makedirs(output_dir, exist_ok=True)
    
    # 2. 获取所有EDF文件
    edf_files = sorted(glob(os.path.join(data_dir, '*.edf')))
    if not edf_files:
        raise FileNotFoundError(f"路径 {data_dir} 下未找到EDF文件")
    print(f"找到{len(edf_files)}个EDF文件:")
    
    # 3. 初始化存储变量
    all_means = []
    all_stds = []
    channel_names = None
    TARGET_CHANNELS = ['F3-C3', 'C3-P3', 'F4-C4', 'C4-P4', 'FZ-CZ', 'CZ-PZ']
    
    # 4. 处理每个文件
    for i, edf_file in enumerate(edf_files, 1):
        try:
            print(f"\n处理文件 {i}/{len(edf_files)}: {os.path.basename(edf_file)}")
            
            # 加载文件（自动处理重复通道名）
            raw = mne.io.read_raw_edf(edf_file, preload=True)
            
            # 首次运行时记录通道信息
            if channel_names is None:
                channel_names = raw.ch_names
                print(f"所有通道 ({len(channel_names)}个): {channel_names}")
            
            # 检查通道一致性
            if raw.ch_names != channel_names:
                print(f"警告: 通道不一致！跳过该文件")
                continue
            
            # 计算PSD（兼容所有MNE版本）
            psds = raw.compute_psd(
                method='welch',
                fmin=1,
                fmax=50,
                n_fft=int(3 * raw.info['sfreq']),  # 3秒窗长
                n_overlap=int(1.5 * raw.info['sfreq'])  # 50%重叠
            )
            
            # 获取实际窗长（新版MNE兼容方案）
            n_freqs = psds.get_data().shape[1]
            effective_window = (n_freqs * 2 - 1) / raw.info['sfreq']
            print(f"PSD计算完成，实际窗长: {effective_window:.2f}秒")
            
            # 保存结果
            all_means.append(psds.get_data().mean(axis=1))
            all_stds.append(psds.get_data().std(axis=1))
            
        except Exception as e:
            print(f"处理失败: {str(e)}")
            continue
    
    # 5. 检查是否有有效数据
    if not all_means:
        raise ValueError("没有文件被成功处理，请检查数据！")
    
    # 6. 计算全局统计量
    global_mean = np.mean(all_means, axis=0)
    global_std = np.mean(all_stds, axis=0)
    
    # 7. 构建结果DataFrame
    df = pd.DataFrame({
        'Channel': channel_names,
        'Global_Mean_PSD': global_mean,
        'Global_Std_PSD': global_std,
        'Is_Target': ['✓' if ch in TARGET_CHANNELS else '' for ch in channel_names]
    })
    
    # 8. 生成双排序结果
    df_mean = df.sort_values('Global_Mean_PSD', ascending=False).reset_index(drop=True)
    df_mean['Rank_Mean'] = df_mean.index + 1
    
    df_std = df.sort_values('Global_Std_PSD', ascending=True).reset_index(drop=True)
    df_std['Rank_Std'] = df_std.index + 1
    
    # 9. 合并最终结果
    final_df = pd.merge(
        df_mean[['Rank_Mean', 'Channel', 'Global_Mean_PSD', 'Is_Target']],
        df_std[['Rank_Std', 'Channel', 'Global_Std_PSD', 'Is_Target']],
        on='Channel'
    ).rename(columns={'Is_Target_x': 'Is_Target'})
    
    # 10. 输出结果
    print("\n" + "="*80)
    print("综合所有文件的通道PSD表现（按均值降序）:")
    display(df_mean.head(20))
    
    print("\n" + "="*80)
    print("综合所有文件的通道PSD表现（按标准差升序）:")
    display(df_std.head(20))
    
    # 11. 保存结果
    csv_path = os.path.join(output_dir, f'chb{subject_id}_psd_ranking.csv')
    final_df.to_csv(csv_path, index=False)
    print(f"\n结果已保存到 {csv_path}")
    
    return final_df

# 使用示例
try:
    results = batch_analyze_channels(subject_id='02')
    print("\n处理完成！目标通道排名：")
    print(results[results['Is_Target'] == '✓'].sort_values('Rank_Mean'))
except Exception as e:
    print(f"运行出错: {str(e)}")

找到36个EDF文件:

处理文件 1/36: chb02_01.edf
Extracting EDF parameters from /root/autodl-tmp/epli/chb02/chb02_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


/tmp/ipykernel_5461/3775168641.py:37: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=True)


所有通道 (23个): ['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8-0', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8-1']
Effective window size : 3.000 (s)
PSD计算完成，实际窗长: 1.15秒

处理文件 2/36: chb02_02.edf
Extracting EDF parameters from /root/autodl-tmp/epli/chb02/chb02_02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


/tmp/ipykernel_5461/3775168641.py:37: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=True)


Effective window size : 3.000 (s)
PSD计算完成，实际窗长: 1.15秒

处理文件 3/36: chb02_03.edf
Extracting EDF parameters from /root/autodl-tmp/epli/chb02/chb02_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


...

Effective window size : 3.000 (s)
PSD计算完成，实际窗长: 1.15秒

综合所有文件的通道PSD表现（按均值降序）:


,Channel,Global_Mean_PSD,Global_Std_PSD,Is_Target,Rank_Mean
0,CZ-PZ,7.210373e-11,1.736218e-10,✓,1
1,FP2-F8,6.661392e-11,2.225689e-10,,2
2,P4-O2,6.403114e-11,1.458593e-10,,3
3,C4-P4,6.401087e-11,1.585871e-10,✓,4
4,FP2-F4,6.229728e-11,2.077198e-10,,5
5,FP1-F7,4.859583e-11,1.508128e-10,,6
6,T7-P7,4.674967e-11,9.167790e-11,,7
7,P7-T7,4.674967e-11,9.167790e-11,,8
8,T8-P8-0,4.637966e-11,7.408105e-11,,9
9,T8-P8-1,4.637966e-11,7.408105e-11,,10



综合所有文件的通道PSD表现（按标准差升序）:


,Channel,Global_Mean_PSD,Global_Std_PSD,Is_Target,Rank_Std
0,P8-O2,2.473940e-11,4.971620e-11,,1
1,F4-C4,2.077055e-11,6.150123e-11,✓,2
2,F8-T8,4.119219e-11,6.448076e-11,,3
3,FT10-T8,3.905140e-11,6.868269e-11,,4
4,T8-P8-1,4.637966e-11,7.408105e-11,,5
5,T8-P8-0,4.637966e-11,7.408105e-11,,6
6,T7-FT9,3.776887e-11,7.503611e-11,,7
7,FZ-CZ,2.711537e-11,7.995633e-11,✓,8
8,F7-T7,4.398983e-11,8.243499e-11,,9
9,P7-O1,2.721781e-11,8.346626e-11,,10



结果已保存到 /root/autodl-tmp/processed/chb02_psd_ranking.csv

处理完成！目标通道排名：
    Rank_Mean Channel  Global_Mean_PSD Is_Target  Rank_Std  Global_Std_PSD  \
0           1   CZ-PZ     7.210373e-11         ✓        21    1.736218e-10   
3           4   C4-P4     6.401087e-11         ✓        20    1.585871e-10   
11         12   C3-P3     4.425327e-11         ✓        15    1.231732e-10   
16         17   F3-C3     3.827610e-11         ✓        14    1.143194e-10   
20         21   FZ-CZ     2.711537e-11         ✓         8    7.995633e-11   
22         23   F4-C4     2.077055e-11         ✓         2    6.150123e-11   

   Is_Target_y  
0            ✓  
3            ✓  
11           ✓  
16           ✓  
20           ✓  
22           ✓  


In [5]:
#
import os
import shutil

def fix_nested_folder(root_dir):
    """
    修复嵌套的chbXX文件夹结构
    例如：/epli/chb01/chb01/ -> /epli/chb01/
    """
    for subject_folder in os.listdir(root_dir):
        if subject_folder.startswith('chb'):
            subject_path = os.path.join(root_dir, subject_folder)
            
            # 检查是否存在嵌套文件夹
            nested_folder = os.path.join(subject_path, subject_folder)
            if os.path.exists(nested_folder):
                print(f"发现嵌套文件夹: {nested_folder}")
                
                # 移动所有文件到外层目录
                for item in os.listdir(nested_folder):
                    src = os.path.join(nested_folder, item)
                    dst = os.path.join(subject_path, item)
                    shutil.move(src, dst)
                
                # 删除空文件夹
                try:
                    os.rmdir(nested_folder)
                    print(f"已修复: {subject_folder} 的嵌套结构")
                except OSError as e:
                    print(f"警告: 无法删除 {nested_folder} - {str(e)}")

# 使用示例
fix_nested_folder('/root/autodl-tmp/epli')

# 然后可以运行之前的分析代码

发现嵌套文件夹: /root/autodl-tmp/epli/chb01/chb01
已修复: chb01 的嵌套结构
发现嵌套文件夹: /root/autodl-tmp/epli/chb13/chb13
已修复: chb13 的嵌套结构
发现嵌套文件夹: /root/autodl-tmp/epli/chb18/chb18
已修复: chb18 的嵌套结构
发现嵌套文件夹: /root/autodl-tmp/epli/chb19/chb19
已修复: chb19 的嵌套结构
发现嵌套文件夹: /root/autodl-tmp/epli/chb20/chb20
已修复: chb20 的嵌套结构
发现嵌套文件夹: /root/autodl-tmp/epli/chb21/chb21
已修复: chb21 的嵌套结构
发现嵌套文件夹: /root/autodl-tmp/epli/chb23/chb23
已修复: chb23 的嵌套结构


In [ ]:
##09总是使得kernal死亡 先不考虑


In [2]:
import os
import numpy as np
import pandas as pd
import mne
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt

# 设置目标通道
TARGET_CHANNELS = ['F3-C3', 'C3-P3', 'F4-C4', 'C4-P4', 'FZ-CZ', 'CZ-PZ']

def calculate_psd_stats(subject_id):
    """计算单个被试所有EDF文件的PSD统计（使用加权综合评分排名）"""
    data_dir = f'/root/autodl-tmp/epli/chb{subject_id}'
    output_dir = '/root/autodl-tmp/processed2'
    os.makedirs(output_dir, exist_ok=True)
    
    edf_files = sorted(glob(os.path.join(data_dir, '*.edf')))
    if not edf_files:
        raise FileNotFoundError(f"被试{subject_id}无EDF文件")
    
    all_psd_means = []
    all_psd_vars = []
    channel_names = None
    
    for edf_file in tqdm(edf_files, desc=f"处理被试{subject_id}"):
        try:
            raw = mne.io.read_raw_edf(edf_file, preload=True)
            
            if channel_names is None:
                channel_names = raw.ch_names
                print(f"\n被试{subject_id}的通道 ({len(channel_names)}个):")
                print(channel_names)
            
            # 计算PSD
            psd = raw.compute_psd(method='welch', fmin=1, fmax=50,
                                n_fft=int(3*raw.info['sfreq']),
                                n_overlap=int(1.5*raw.info['sfreq']))
            
            all_psd_means.append(psd.get_data().mean(axis=1))
            all_psd_vars.append(psd.get_data().var(axis=1))
            
        except Exception as e:
            print(f"\n文件{os.path.basename(edf_file)}处理失败: {str(e)}")
            continue
    
    if not all_psd_means:
        raise ValueError(f"被试{subject_id}无有效数据")
    
    # 计算全局统计量
    global_mean = np.mean(all_psd_means, axis=0)
    global_var = np.mean(all_psd_vars, axis=0)
    global_std = np.sqrt(global_var)
    
    # 创建结果DataFrame
    df = pd.DataFrame({
        'Channel': channel_names,
        'Mean_PSD': global_mean,
        'Variance_PSD': global_var,
        'Std_PSD': global_std,
        'Is_Target': ['✓' if ch in TARGET_CHANNELS else '' for ch in channel_names]
    })
    
    # 加权综合评分排名
    df['Norm_Mean'] = (df['Mean_PSD'] - df['Mean_PSD'].min()) / \
                     (df['Mean_PSD'].max() - df['Mean_PSD'].min())
    df['Norm_Var'] = 1 - (df['Variance_PSD'] - df['Variance_PSD'].min()) / \
                   (df['Variance_PSD'].max() - df['Variance_PSD'].min())
    df['Composite_Score'] = 0.5 * df['Norm_Mean'] + 0.5 * df['Norm_Var']
    df_sorted = df.sort_values('Composite_Score', ascending=False)
    df_sorted['Rank'] = range(1, len(df_sorted)+1)
    
    # 保存结果（每个被试单独保存）
    csv_path = os.path.join(output_dir, f'chb{subject_id}_psd_results.csv')
    df_sorted.to_csv(csv_path, index=False)
    print(f"被试{subject_id}结果已保存到: {csv_path}")
    
    return df_sorted

def batch_process_subjects(subject_ids, batch_size=3):
    """批量处理被试（不合并结果）"""
    all_results = []
    
    for i in range(0, len(subject_ids), batch_size):
        batch = subject_ids[i:i+batch_size]
        print(f"\n=== 处理批次: {batch} ===")
        
        for subject_id in batch:
            try:
                result = calculate_psd_stats(subject_id)
                all_results.append(result)
                
                # 打印目标通道信息
                target_result = result[result['Is_Target'] == '✓']
                print(f"\n被试{subject_id}目标通道综合评分排名:")
                print(target_result[['Rank', 'Channel', 'Composite_Score', 
                                   'Mean_PSD', 'Variance_PSD']])
                
            except Exception as e:
                print(f"\n被试{subject_id}处理失败: {str(e)}")
                continue
    
    # 生成汇总报告（基于所有被试的单独文件）
    generate_summary_report(output_dir='/root/autodl-tmp/processed2')
    
    return all_results

def generate_summary_report(output_dir):
    """从已保存的单个文件中生成汇总报告"""
    # 获取所有被试结果文件
    result_files = sorted(glob(os.path.join(output_dir, 'chb*_psd_results.csv')))
    
    if not result_files:
        print("没有找到结果文件！")
        return
    
    # 读取并汇总所有目标通道数据
    target_dfs = []
    for file in result_files:
        try:
            df = pd.read_csv(file)
            target_df = df[df['Is_Target'] == '✓'].copy()
            subject_id = os.path.basename(file).split('_')[0][3:]
            target_df['Subject'] = subject_id
            target_dfs.append(target_df)
        except Exception as e:
            print(f"读取文件{file}失败: {str(e)}")
    
    if not target_dfs:
        print("没有有效目标通道数据！")
        return
    
    all_target = pd.concat(target_dfs)
    
    # 计算汇总统计
    summary = all_target.groupby('Channel').agg({
        'Rank': ['mean', 'min', 'max'],
        'Composite_Score': 'mean',
        'Mean_PSD': 'mean',
        'Variance_PSD': 'mean'
    }).reset_index()
    
    # 重命名列
    summary.columns = ['Channel', 'Avg_Rank', 'Min_Rank', 'Max_Rank', 
                      'Avg_Composite_Score', 'Avg_PSD', 'Avg_Variance']
    summary = summary.sort_values('Avg_Composite_Score', ascending=False)
    
    # 保存报告
    report_file = os.path.join(output_dir, 'summary_report.csv')
    summary.to_csv(report_file, index=False)
    
    print("\n目标通道综合评分汇总报告:")
    print(summary)
    
    # 可视化
    plt.figure(figsize=(12, 6))
    plt.subplot(121)
    plt.bar(summary['Channel'], summary['Avg_Composite_Score'], color='skyblue')
    plt.title('目标通道平均综合评分')
    plt.xticks(rotation=45)
    plt.ylabel('综合评分')
    
    plt.subplot(122)
    plt.bar(summary['Channel'], summary['Avg_PSD'], color='lightgreen')
    plt.title('目标通道平均PSD值')
    plt.xticks(rotation=45)
    plt.ylabel('PSD值')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'target_channels_summary.png'))
    plt.close()

# 使用示例
if __name__ == "__main__":
    # 所有被试列表（排除缺失编号）
    all_subjects = [f"{i:02d}" for i in range(1, 24) if i not in [1,2,3,4,5,6,7,8,9,11,12,14,15,16,17,20,21,22,23]]
    print("开始批量处理（使用加权综合评分排名）...")
    results = batch_process_subjects(all_subjects, batch_size=3)
    print("\n处理完成！所有结果已单独保存")

开始批量处理（使用加权综合评分排名）...

=== 处理批次: ['10', '13', '18'] ===


处理被试10:   0%|          | 0/25 [00:00<?, ?it/s]

Extracting EDF parameters from /root/autodl-tmp/epli/chb10/chb10_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1848319  =      0.000 ...  7219.996 secs...


/tmp/ipykernel_2441/2022054938.py:28: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=True)



被试10的通道 (23个):
['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8-0', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8-1']
Effective window size : 3.000 (s)


...

Effective window size : 3.000 (s)


/tmp/ipykernel_2441/2022054938.py:36: UserWarning: Zero value in spectrum for channels --0, --1, --2, --3, --4
  psd = raw.compute_psd(method='welch', fmin=1, fmax=50,
处理被试19: 100%|██████████| 30/30 [01:11<00:00,  2.38s/it]



被试19处理失败: operands could not be broadcast together with shapes (22,) (28,) 

目标通道综合评分汇总报告:
  Channel   Avg_Rank  Min_Rank  Max_Rank  Avg_Composite_Score       Avg_PSD  \
3   F3-C3   9.222222         2        17             0.593870  4.045895e-10   
4   F4-C4  11.333333         2        19             0.564612  3.921805e-10   
2   CZ-PZ  13.111111         1        23             0.554560  4.037137e-10   
0   C3-P3  13.555556         4        22             0.553730  3.879536e-10   
5   FZ-CZ  14.555556         1        26             0.528570  4.075702e-10   
1   C4-P4  16.444444         5        23             0.520237  3.886761e-10   

   Avg_Variance  
3  3.324758e-17  
4  3.321930e-17  
2  3.400680e-17  
0  3.332578e-17  
5  3.403129e-17  
1  3.380898e-17  


/tmp/ipykernel_2441/2022054938.py:169: UserWarning: Glyph 32508 (\N{CJK UNIFIED IDEOGRAPH-7EFC}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_2441/2022054938.py:169: UserWarning: Glyph 21512 (\N{CJK UNIFIED IDEOGRAPH-5408}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_2441/2022054938.py:169: UserWarning: Glyph 35780 (\N{CJK UNIFIED IDEOGRAPH-8BC4}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_2441/2022054938.py:169: UserWarning: Glyph 20998 (\N{CJK UNIFIED IDEOGRAPH-5206}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_2441/2022054938.py:169: UserWarning: Glyph 30446 (\N{CJK UNIFIED IDEOGRAPH-76EE}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_2441/2022054938.py:169: UserWarning: Glyph 26631 (\N{CJK UNIFIED IDEOGRAPH-6807}) missing from current font.
  plt.tight_layout()
/tmp/ipykernel_2441/2022054938.py:169: UserWarning: Glyph 36890 (\N{CJK UNIFIED IDEOGRAPH-901A}) missing from current font.
  plt.


处理完成！所有结果已单独保存


In [3]:
import os
import mne
from glob import glob

def check_subject_channels(subject_ids):
    """
    检查指定被试的EDF文件通道信息
    参数：
        subject_ids: 被试ID列表，如 ['13', '18', '19']
    """
    base_dir = '/root/autodl-tmp/epli'  # 根据实际路径调整
    
    for subject_id in subject_ids:
        print(f"\n=== 检查被试 chb{subject_id} ===")
        subject_dir = os.path.join(base_dir, f'chb{subject_id}')
        
        # 获取所有EDF文件
        edf_files = sorted(glob(os.path.join(subject_dir, '*.edf')))
        
        if not edf_files:
            print(f"警告: 被试 chb{subject_id} 目录下未找到EDF文件")
            continue
            
        print(f"找到 {len(edf_files)} 个EDF文件")
        
        # 检查第一个文件的通道信息（假设同被试文件通道一致）
        first_file = edf_files[0]
        try:
            # 快速读取文件头信息（不加载数据）
            raw = mne.io.read_raw_edf(first_file, preload=False)
            
            # 打印通道信息
            print(f"\n文件: {os.path.basename(first_file)}")
            print(f"采样率: {raw.info['sfreq']} Hz")
            print(f"通道数量: {len(raw.ch_names)}")
            print("通道列表:")
            for i, ch in enumerate(raw.ch_names, 1):
                print(f"{i:2d}. {ch}")
                
            # 检查重复通道名
            duplicates = set([ch for ch in raw.ch_names if raw.ch_names.count(ch) > 1])
            if duplicates:
                print(f"\n警告: 发现重复通道名 - {duplicates}")
                
            # 检查特殊标记通道（如--0, --1等）
            auto_channels = [ch for ch in raw.ch_names if ch.startswith('--')]
            if auto_channels:
                print(f"警告: 存在自动命名通道 - {auto_channels}")
                
        except Exception as e:
            print(f"读取文件 {os.path.basename(first_file)} 失败: {str(e)}")
            continue
            
        # 可选：验证其他文件的通道一致性
        if len(edf_files) > 1:
            print("\n验证其他文件的通道一致性...")
            for edf_file in edf_files[1:3]:  # 只检查前3个文件以节省时间
                try:
                    raw = mne.io.read_raw_edf(edf_file, preload=False)
                    if set(raw.ch_names) != set(raw.ch_names):
                        print(f"文件 {os.path.basename(edf_file)} 通道不一致")
                except:
                    continue

# 使用示例
if __name__ == "__main__":
    subjects_to_check = ['13', '18', '19']
    print(f"开始检查被试 {', '.join(subjects_to_check)} 的通道信息...")
    check_subject_channels(subjects_to_check)
    print("\n检查完成！")

开始检查被试 13, 18, 19 的通道信息...

=== 检查被试 chb13 ===
找到 33 个EDF文件
Extracting EDF parameters from /root/autodl-tmp/epli/chb13/chb13_02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...

文件: chb13_02.edf
采样率: 256.0 Hz
通道数量: 28
通道列表:
 1. FP1-F7
 2. F7-T7
 3. T7-P7
 4. P7-O1
 5. --0
 6. FP1-F3
 7. F3-C3
 8. C3-P3
 9. P3-O1
10. --1
11. FZ-CZ
12. CZ-PZ
13. --2
14. FP2-F4
15. F4-C4
16. C4-P4
17. P4-O2
18. --3
19. FP2-F8
20. F8-T8
21. T8-P8-0
22. P8-O2
23. --4
24. P7-T7
25. T7-FT9
26. FT9-FT10
27. FT10-T8
28. T8-P8-1
警告: 存在自动命名通道 - ['--0', '--1', '--2', '--3', '--4']

验证其他文件的通道一致性...
Extracting EDF parameters from /root/autodl-tmp/epli/chb13/chb13_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /root/autodl-tmp/epli/chb13/chb13_04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...

=== 检查被试 chb18 ===
找到 36 个EDF文件
Extracting EDF parameters from /ro

/tmp/ipykernel_2441/138278005.py:30: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(first_file, preload=False)
/tmp/ipykernel_2441/138278005.py:30: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  raw = mne.io.read_raw_edf(first_file, preload=False)
/tmp/ipykernel_2441/138278005.py:59: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=False)
/tmp/ipykernel_2441/138278005.py:59: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  raw = mne.io.read_raw_edf(edf_file, preload=False)
/tmp/ipykernel_2441/138278005.py:59: RuntimeWarning: Channel names are not unique, found duplicates for: {'-'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=F


16. C4-P4
17. P4-O2
18. --3
19. FP2-F8
20. F8-T8
21. T8-P8
22. P8-O2
警告: 存在自动命名通道 - ['--0', '--1', '--2', '--3']

验证其他文件的通道一致性...
Extracting EDF parameters from /root/autodl-tmp/epli/chb18/chb18_02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /root/autodl-tmp/epli/chb18/chb18_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...

=== 检查被试 chb19 ===
找到 30 个EDF文件
Extracting EDF parameters from /root/autodl-tmp/epli/chb19/chb19_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...

文件: chb19_01.edf
采样率: 256.0 Hz
通道数量: 22
通道列表:
 1. FP1-F7
 2. F7-T7
 3. T7-P7
 4. P7-O1
 5. --0
 6. FP1-F3
 7. F3-C3
 8. C3-P3
 9. P3-O1
10. --1
11. FZ-CZ
12. CZ-PZ
13. --2
14. FP2-F4
15. F4-C4
16. C4-P4
17. P4-O2
18. --3
19. FP2-F8
20. F8-T8
21. T8-P8
22. P8-O2
警告: 存在自动命名通道 - ['--0', '--1', '--2', '--3']

验证其他文件的通道一致性...
Extracting EDF parameters from /root/a

/tmp/ipykernel_2441/138278005.py:59: RuntimeWarning: Channel names are not unique, found duplicates for: {'.', 'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=False)
/tmp/ipykernel_2441/138278005.py:59: RuntimeWarning: Channel names are not unique, found duplicates for: {'.', 'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=False)
/tmp/ipykernel_2441/138278005.py:30: RuntimeWarning: Channel names are not unique, found duplicates for: {'-'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(first_file, preload=False)
/tmp/ipykernel_2441/138278005.py:30: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3
  raw = mne.io.read_raw_edf(first_file, preload=False)


Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /root/autodl-tmp/epli/chb19/chb19_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...

检查完成！


/tmp/ipykernel_2441/138278005.py:59: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=False)
/tmp/ipykernel_2441/138278005.py:59: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  raw = mne.io.read_raw_edf(edf_file, preload=False)
/tmp/ipykernel_2441/138278005.py:59: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=False)
/tmp/ipykernel_2441/138278005.py:59: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  raw = mne.io.read_raw_edf(edf_file, preload=False)


In [4]:
import os
import numpy as np
import pandas as pd
import mne
from glob import glob
import matplotlib.pyplot as plt

def check_auto_named_channels(subject_ids):
    """
    检查指定被试的自动命名通道数据是否为空
    参数：
        subject_ids: 被试ID列表，如 ['13', '18', '19']
    """
    base_dir = '/root/autodl-tmp/epli'
    results = []
    
    for subject_id in subject_ids:
        print(f"\n=== 分析被试 chb{subject_id} ===")
        subject_dir = os.path.join(base_dir, f'chb{subject_id}')
        edf_files = sorted(glob(os.path.join(subject_dir, '*.edf')))
        
        if not edf_files:
            print(f"警告: 被试 chb{subject_id} 目录下未找到EDF文件")
            continue
            
        # 只分析每个被试的第一个文件（如需全部分析可移除[0]）
        for edf_file in [edf_files[0]]:  
            file_name = os.path.basename(edf_file)
            print(f"\n分析文件: {file_name}")
            
            try:
                # 1. 读取EDF文件（不预加载数据）
                raw = mne.io.read_raw_edf(edf_file, preload=False)
                
                # 2. 识别自动命名通道
                auto_channels = [ch for ch in raw.ch_names if ch.startswith('--')]
                
                if not auto_channels:
                    print("未发现自动命名通道")
                    continue
                    
                print(f"发现自动命名通道: {auto_channels}")
                
                # 3. 检查通道数据
                channel_stats = []
                for ch in auto_channels:
                    # 加载通道数据（仅加载目标通道节省内存）
                    raw_ch = raw.copy().pick_channels([ch])
                    data = raw_ch.get_data().flatten()
                    
                    # 计算统计指标
                    stats = {
                        'Subject': f'chb{subject_id}',
                        'File': file_name,
                        'Channel': ch,
                        'Max_Abs': np.max(np.abs(data)),
                        'Mean': np.mean(data),
                        'Std': np.std(data),
                        'Is_Empty': np.max(np.abs(data)) < 1e-6,  # 阈值设为1微伏
                        'N_Zero': np.sum(np.abs(data) < 1e-6) / len(data)  # 零值比例
                    }
                    channel_stats.append(stats)
                
                # 4. 保存结果
                df = pd.DataFrame(channel_stats)
                results.append(df)
                
                # 5. 打印结果
                print("\n自动通道数据分析结果:")
                print(df[['Channel', 'Max_Abs', 'Is_Empty', 'N_Zero']])
                
                # 6. 可视化（可选）
                plot_channel_data(raw, auto_channels, subject_id, file_name)
                
            except Exception as e:
                print(f"分析失败: {str(e)}")
                continue
    
    # 合并所有结果
    if results:
        final_df = pd.concat(results)
        save_path = '/root/autodl-tmp/processed/auto_channels_report.csv'
        final_df.to_csv(save_path, index=False)
        print(f"\n结果已保存到: {save_path}")
        return final_df
    else:
        print("\n未发现有效的自动命名通道数据")
        return None

def plot_channel_data(raw, channels, subject_id, file_name):
    """绘制通道数据可视化"""
    plt.figure(figsize=(12, 6))
    
    # 随机选取5秒数据展示
    t_min, t_max = 0, 5  
    sfreq = raw.info['sfreq']
    n_samples = int((t_max - t_min) * sfreq)
    
    for i, ch in enumerate(channels, 1):
        # 加载通道数据
        data = raw.copy().crop(t_min, t_max).pick_channels([ch]).get_data().flatten()
        times = np.linspace(t_min, t_max, len(data))
        
        plt.subplot(len(channels), 1, i)
        plt.plot(times, data, label=ch)
        plt.ylabel('Amplitude (μV)')
        plt.legend()
        
        # 标记零值区域
        zero_mask = np.abs(data) < 1e-6
        if np.any(zero_mask):
            plt.fill_between(times, -10, 10, where=zero_mask, 
                            color='red', alpha=0.2, label='Zero Values')
    
    plt.xlabel('Time (s)')
    plt.suptitle(f'Subject {subject_id} - {file_name}\nAuto-named Channels Data')
    plt.tight_layout()
    
    # 保存图像
    plot_path = f'/root/autodl-tmp/processed/chb{subject_id}_auto_channels.png'
    plt.savefig(plot_path)
    plt.close()
    print(f"通道数据图已保存到: {plot_path}")

# 使用示例
if __name__ == "__main__":
    subjects = ['13', '18', '19']
    result_df = check_auto_named_channels(subjects)
    
    if result_df is not None:
        print("\n汇总统计:")
        print(result_df.groupby('Subject').agg({
            'Channel': 'count',
            'Max_Abs': ['min', 'max', 'mean'],
            'Is_Empty': 'mean',
            'N_Zero': 'mean'
        }))


=== 分析被试 chb13 ===

分析文件: chb13_02.edf
Extracting EDF parameters from /root/autodl-tmp/epli/chb13/chb13_02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
发现自动命名通道: ['--0', '--1', '--2', '--3', '--4']
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


/tmp/ipykernel_2441/2004231775.py:33: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=False)
/tmp/ipykernel_2441/2004231775.py:33: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  raw = mne.io.read_raw_edf(edf_file, preload=False)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).

自动通道数据分析结果:
  Channel   Max_Abs  Is_Empty  N_Zero
0     --0  0.000001     False     0.0
1     --1  0.000001     False     0.0
2     --2  0.000001     False     0.0
3     --3  0.000001     False     0.0
4     --4  0.000001     False     0.0
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
通道数据图已保存到: /root/autodl-tmp/processed/c

/tmp/ipykernel_2441/2004231775.py:33: RuntimeWarning: Channel names are not unique, found duplicates for: {'-'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=False)
/tmp/ipykernel_2441/2004231775.py:33: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3
  raw = mne.io.read_raw_edf(edf_file, preload=False)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).

自动通道数据分析结果:
  Channel   Max_Abs  Is_Empty  N_Zero
0     --0  0.000001     False     0.0
1     --1  0.000001     False     0.0
2     --2  0.000001     False     0.0
3     --3  0.000001     False     0.0
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
通道数据图已保存到: /root/autodl-tmp/processed/chb18_auto_channels.png

=== 分析被试 chb19 ===

分析文件: chb19_01.edf
Extracting EDF parameters from /root/autodl-tmp/epli/chb19/chb19_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
发现自动命名通道: ['--0', '--1', '--2', '--3']
NOTE: pick_channel

/tmp/ipykernel_2441/2004231775.py:33: RuntimeWarning: Channel names are not unique, found duplicates for: {'-'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=False)
/tmp/ipykernel_2441/2004231775.py:33: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3
  raw = mne.io.read_raw_edf(edf_file, preload=False)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).

自动通道数据分析结果:
  Channel   Max_Abs  Is_Empty  N_Zero
0     --0  0.000001     False     0.0
1     --1  0.000001     False     0.0
2     --2  0.000001     False     0.0
3     --3  0.000001     False     0.0
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
通道数据图已保存到: /root/autodl-tmp/processed/chb19_auto_channels.png

结果已保存到: /root/autodl-tmp/processed/auto_channels_report.csv

汇总统计:
        Channel   Max_Abs                     Is_Empty N_Zero
          count       min       max      mean     mean   mean
Subject                                                      
c

In [ ]:
#处理异常的13 18 19 09被试
import os
import mne
from glob import glob
import pandas as pd
from collections import defaultdict

def get_all_subjects_channels(subject_ids):
    """
    获取多个被试的所有通道信息并进行对比分析
    参数：
        subject_ids: 被试ID列表，如 ['01', '13', '18', '19']
    返回：
        tuple: (所有被试通道数据, 异常通道报告)
    """
    base_dir = '/root/autodl-tmp/epli'
    all_channels_data = {}
    
    # 获取所有被试的通道信息
    for subject_id in subject_ids:
        subject_dir = os.path.join(base_dir, f'chb{subject_id}')
        edf_files = sorted(glob(os.path.join(subject_dir, '*.edf')))
        
        if not edf_files:
            print(f"警告: 被试 chb{subject_id} 无EDF文件")
            continue
            
        # 读取第一个文件的通道信息（假设同被试通道一致）
        try:
            raw = mne.io.read_raw_edf(edf_files[0], preload=False)
            all_channels_data[f'chb{subject_id}'] = {
                'channels': raw.ch_names,
                'n_channels': len(raw.ch_names),
                'sample_file': os.path.basename(edf_files[0])
            }
        except Exception as e:
            print(f"读取被试 chb{subject_id} 失败: {str(e)}")
            continue
    
    # 生成异常通道报告（对比13、18、19与01的差异）
    reference_subject = 'chb01'
    anomaly_report = {}
    
    if reference_subject in all_channels_data:
        ref_channels = set(all_channels_data[reference_subject]['channels'])
        
        for subj in ['chb13', 'chb18', 'chb19']:
            if subj in all_channels_data:
                current_channels = set(all_channels_data[subj]['channels'])
                
                # 找出异常通道（存在差异的通道）
                extra_channels = current_channels - ref_channels
                missing_channels = ref_channels - current_channels
                common_channels = current_channels & ref_channels
                
                anomaly_report[subj] = {
                    'extra_channels': sorted(extra_channels),
                    'missing_channels': sorted(missing_channels),
                    'common_channels_count': len(common_channels),
                    'channel_diff_count': len(extra_channels) + len(missing_channels)
                }
    
    return all_channels_data, anomaly_report

def print_channel_report(all_data, anomalies):
    """打印详细的通道对比报告"""
    print("\n" + "="*50)
    print("各被试通道信息汇总")
    print("="*50)
    
    # 打印基本信息
    for subj, info in all_data.items():
        print(f"\n{subj}:")
        print(f"  示例文件: {info['sample_file']}")
        print(f"  通道总数: {info['n_channels']}")
        print(f"  前10个通道: {info['channels'][:10]}")
    
    # 打印异常分析
    if anomalies:
        print("\n" + "="*50)
        print("异常通道分析（对比chb01）")
        print("="*50)
        
        for subj, report in anomalies.items():
            print(f"\n{subj}异常情况:")
            print(f"  特有通道 ({len(report['extra_channels'])}个):")
            for ch in report['extra_channels']:
                print(f"    - {ch}")
            
            print(f"\n  缺失通道 ({len(report['missing_channels'])}个):")
            for ch in report['missing_channels']:
                print(f"    - {ch}")
            
            print(f"\n  共同通道数: {report['common_channels_count']}")
            print(f"  总差异通道数: {report['channel_diff_count']}")

# 使用示例
if __name__ == "__main__":
    target_subjects = ['01', '13', '18', '19']
    print(f"开始分析被试 {', '.join(target_subjects)} 的通道信息...")
    
    # 获取数据
    channels_data, anomaly_report = get_all_subjects_channels(target_subjects)
    
    # 打印报告
    print_channel_report(channels_data, anomaly_report)
    
    # 保存结果到CSV
    output_dir = '/root/autodl-tmp/processed2'
    os.makedirs(output_dir, exist_ok=True)
    
    # 保存各被试通道列表
    for subj, info in channels_data.items():
        df = pd.DataFrame({
            'Channel': info['channels'],
            'Position': range(1, len(info['channels'])+1),
            'Is_Reference': ['Yes' if subj == 'chb01' else 'No'] * len(info['channels'])
        })
        df.to_csv(os.path.join(output_dir, f'{subj}_full_channels.csv'), index=False)
    
    # 保存异常报告
    if anomaly_report:
        anomaly_df = pd.DataFrame.from_dict(anomaly_report, orient='index')
        anomaly_df.to_csv(os.path.join(output_dir, 'channel_anomalies_report.csv'))
        print(f"\n异常报告已保存到: {os.path.join(output_dir, 'channel_anomalies_report.csv')}")

In [4]:
##为了便于选择通道 先看均有的通道
import os
import mne
from glob import glob

def get_common_channels_list(subject_ids):
    """
    获取多个被试共有的通道列表
    参数：
        subject_ids: 被试ID列表，如 ['01', '13', '18', '19']
    返回：
        list: 所有被试共有的通道名称列表（按字母排序）
    """
    base_dir = '/root/autodl-tmp/epli'
    channel_sets = []
    
    for subject_id in subject_ids:
        subject_dir = os.path.join(base_dir, f'chb{subject_id}')
        edf_files = sorted(glob(os.path.join(subject_dir, '*.edf')))
        
        if not edf_files:
            print(f"警告: 被试 chb{subject_id} 无EDF文件")
            continue
            
        try:
            # 读取第一个文件的通道信息
            raw = mne.io.read_raw_edf(edf_files[0], preload=False)
            channel_sets.append(set(raw.ch_names))
            print(f"被试 chb{subject_id} 通道数: {len(raw.ch_names)}")
        except Exception as e:
            print(f"读取被试 chb{subject_id} 失败: {str(e)}")
            continue
    
    # 计算所有被试共有的通道（按字母排序）
    if len(channel_sets) > 1:
        common_channels = sorted(set.intersection(*channel_sets))
        return common_channels
    else:
        print("错误: 至少需要两个有效被试才能计算共有通道")
        return []

# 使用示例
if __name__ == "__main__":
    target_subjects = ['01', '13', '18', '19']
    common_channels = get_common_channels_list(target_subjects)
    
    # 直接打印列表
    print("\n=== 四个被试共有的通道列表 ===")
    print(common_channels)
    
    # 如需美化输出
    print("\n格式化输出：")
    for i, ch in enumerate(common_channels, 1):
        print(f"{i}. {ch}")

Extracting EDF parameters from /root/autodl-tmp/epli/chb01/chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
被试 chb01 通道数: 23
Extracting EDF parameters from /root/autodl-tmp/epli/chb13/chb13_02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
被试 chb13 通道数: 28
Extracting EDF parameters from /root/autodl-tmp/epli/chb18/chb18_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
被试 chb18 通道数: 22
Extracting EDF parameters from /root/autodl-tmp/epli/chb19/chb19_01.edf...


/tmp/ipykernel_2985/3850851840.py:27: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_files[0], preload=False)
/tmp/ipykernel_2985/3850851840.py:27: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_files[0], preload=False)
/tmp/ipykernel_2985/3850851840.py:27: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  raw = mne.io.read_raw_edf(edf_files[0], preload=False)
/tmp/ipykernel_2985/3850851840.py:27: RuntimeWarning: Channel names are not unique, found duplicates for: {'-'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_files[0], preload=False)
/tmp/ipykernel_2985/3850851840.py:27: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3
  raw = mne.io.read_raw_edf(edf_files[0]

EDF file detected
Setting channel info structure...
Creating raw.info structure...
被试 chb19 通道数: 22

=== 四个被试共有的通道列表 ===
['C3-P3', 'C4-P4', 'CZ-PZ', 'F3-C3', 'F4-C4', 'F7-T7', 'F8-T8', 'FP1-F3', 'FP1-F7', 'FP2-F4', 'FP2-F8', 'FZ-CZ', 'P3-O1', 'P4-O2', 'P7-O1', 'P8-O2', 'T7-P7']

格式化输出：
1. C3-P3
2. C4-P4
3. CZ-PZ
4. F3-C3
5. F4-C4
6. F7-T7
7. F8-T8
8. FP1-F3
9. FP1-F7
10. FP2-F4
11. FP2-F8
12. FZ-CZ
13. P3-O1
14. P4-O2
15. P7-O1
16. P8-O2
17. T7-P7


/tmp/ipykernel_2985/3850851840.py:27: RuntimeWarning: Channel names are not unique, found duplicates for: {'-'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_files[0], preload=False)
/tmp/ipykernel_2985/3850851840.py:27: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3
  raw = mne.io.read_raw_edf(edf_files[0], preload=False)


ps: 09被试数据量过大+13、18、19被试通道缺失  所以没有考虑pst信息

In [7]:
#read data
import os

# 指定/tmp目录
tmp_dir = '/root/autodl-tmp/processed2'

# 列出所有文件和子目录
contents = os.listdir(tmp_dir)

print(f"/tmp目录下的内容：")
for item in contents:
    print(item)

/tmp目录下的内容：
.ipynb_checkpoints
chb01_psd_results.csv
chb02_psd_results.csv
chb03_psd_results.csv
chb05_psd_results.csv
chb10_psd_results.csv
chb14_psd_results.csv
chb20_psd_results.csv
chb21_psd_results.csv
chb23_psd_results.csv


In [8]:
import os
import pandas as pd
import numpy as np
from glob import glob

# 配置参数
target_dir = '/root/autodl-tmp/processed2'
target_channels = [
    'C3-P3', 'C4-P4', 'CZ-PZ', 'F3-C3', 'F4-C4', 'F7-T7', 'F8-T8',
    'FP1-F3', 'FP1-F7', 'FP2-F4', 'FP2-F8', 'FZ-CZ', 'P3-O1',
    'P4-O2', 'P7-O1', 'P8-O2', 'T7-P7'
]

def analyze_composite_scores():
    """综合分析所有被试的Composite_Score"""
    # 1. 收集所有CSV文件
    csv_files = glob(os.path.join(target_dir, 'chb*_psd_results.csv'))
    if not csv_files:
        raise FileNotFoundError(f"目录 {target_dir} 中未找到CSV文件")
    
    print(f"找到 {len(csv_files)} 个数据文件")
    
    # 2. 初始化存储结构
    channel_scores = {ch: [] for ch in target_channels}
    
    # 3. 处理每个文件
    for file in csv_files:
        try:
            df = pd.read_csv(file)
            subject_id = os.path.basename(file).split('_')[0]
            
            # 筛选目标通道
            target_df = df[df['Channel'].isin(target_channels)]
            
            # 收集各通道分数
            for ch in target_channels:
                if ch in df['Channel'].values:
                    ch_score = df.loc[df['Channel'] == ch, 'Composite_Score'].values[0]
                    channel_scores[ch].append(ch_score)
                else:
                    print(f"警告: 被试 {subject_id} 缺少通道 {ch}")
                    
        except Exception as e:
            print(f"处理文件 {os.path.basename(file)} 失败: {str(e)}")
            continue
    
    # 4. 计算各通道平均分
    result = []
    for ch, scores in channel_scores.items():
        if scores:  # 只处理有数据的通道
            result.append({
                'Channel': ch,
                'Mean_Score': np.mean(scores),
                'N_Subjects': len(scores),
                'Std_Score': np.std(scores) if len(scores) > 1 else 0
            })
    
    # 5. 创建DataFrame并排序
    result_df = pd.DataFrame(result)
    result_df = result_df.sort_values('Mean_Score', ascending=False)
    
    # 6. 转换为最终列表格式
    final_list = [
        (row['Channel'], row['Mean_Score']) 
        for _, row in result_df.iterrows()
    ]
    
    return final_list

# 执行分析
if __name__ == "__main__":
    try:
        sorted_channels = analyze_composite_scores()
        
        print("\n=== 通道综合评分排序 ===")
        print(f"{'通道':<10} {'平均分':<8}")
        print("-" * 20)
        for ch, score in sorted_channels:
            print(f"{ch:<10} {score:.4f}")
        
        print("\n最终排序列表:")
        print([ch for ch, _ in sorted_channels])
        
    except Exception as e:
        print(f"分析失败: {str(e)}")

找到 9 个数据文件

=== 通道综合评分排序 ===
通道         平均分     
--------------------
F7-T7      0.6591
FP1-F3     0.6350
FP2-F4     0.6046
T7-P7      0.5983
F3-C3      0.5939
F8-T8      0.5709
F4-C4      0.5646
FP1-F7     0.5632
P7-O1      0.5615
P3-O1      0.5550
CZ-PZ      0.5546
C3-P3      0.5537
FP2-F8     0.5486
P8-O2      0.5401
FZ-CZ      0.5286
C4-P4      0.5202
P4-O2      0.4962

最终排序列表:
['F7-T7', 'FP1-F3', 'FP2-F4', 'T7-P7', 'F3-C3', 'F8-T8', 'F4-C4', 'FP1-F7', 'P7-O1', 'P3-O1', 'CZ-PZ', 'C3-P3', 'FP2-F8', 'P8-O2', 'FZ-CZ', 'C4-P4', 'P4-O2']


In [9]:
#不考虑前额F1 F2
['F7-T7', 'T7-P7', 'F3-C3', 'F8-T8', 'F4-C4',  'P7-O1', 'P3-O1', 'CZ-PZ', 'C3-P3',  'P8-O2', 'FZ-CZ', 'C4-P4', 'P4-O2']

['F7-T7',
 'T7-P7',
 'F3-C3',
 'F8-T8',
 'F4-C4',
 'P7-O1',
 'P3-O1',
 'CZ-PZ',
 'C3-P3',
 'P8-O2',
 'FZ-CZ',
 'C4-P4',
 'P4-O2']